In [2]:
import torch
import torchvision
import matplotlib
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
train=pd.read_csv("train.csv",index_col=False)

In [22]:
x=list(train.columns)[:-1]
y=train['Insect']

In [23]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    #for col in categorical_cols:
        #dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[x].to_numpy()
    targets_array = dataframe1["Insect"].to_numpy()
    targets_array= targets_array.reshape(-1, 1)
    return inputs_array, targets_array

In [24]:
inputs_array, targets_array = dataframe_to_arrays(train)
inputs_array, targets_array

(array([[ 0.00000000e+00,  2.20000000e+01,  2.60000000e+01, ...,
          2.50195048e+02, -5.35477774e+01, -1.12049983e+02],
        [ 1.00000000e+00,  1.00000000e+01,  1.60000000e+01, ...,
          9.53949414e+01,  3.72919441e+01,  4.85251800e+01],
        [ 2.00000000e+00,  2.10000000e+01,  4.20000000e+01, ...,
         -9.07503862e+01, -5.04765557e+01,  8.53990087e+01],
        ...,
        [ 6.99800000e+03,  2.20000000e+01,  4.70000000e+01, ...,
          2.48745127e+02,  1.16893077e+02,  3.28061501e+01],
        [ 6.99900000e+03,  1.80000000e+01,  1.40000000e+01, ...,
          3.87285646e+01, -3.90504657e+01,  2.03567109e+01],
        [ 7.00000000e+03,  2.10000000e+01,  3.10000000e+01, ...,
         -7.26963849e+01,  1.45299227e+02, -1.43856243e+02]]),
 array([[2],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1]], dtype=int64))

In [25]:
inputs = torch.from_numpy(inputs_array).type(torch.float)
targets = torch.from_numpy(targets_array).type(torch.float)
inputs,targets

(tensor([[ 0.0000e+00,  2.2000e+01,  2.6000e+01,  ...,  2.5020e+02,
          -5.3548e+01, -1.1205e+02],
         [ 1.0000e+00,  1.0000e+01,  1.6000e+01,  ...,  9.5395e+01,
           3.7292e+01,  4.8525e+01],
         [ 2.0000e+00,  2.1000e+01,  4.2000e+01,  ..., -9.0750e+01,
          -5.0477e+01,  8.5399e+01],
         ...,
         [ 6.9980e+03,  2.2000e+01,  4.7000e+01,  ...,  2.4875e+02,
           1.1689e+02,  3.2806e+01],
         [ 6.9990e+03,  1.8000e+01,  1.4000e+01,  ...,  3.8729e+01,
          -3.9050e+01,  2.0357e+01],
         [ 7.0000e+03,  2.1000e+01,  3.1000e+01,  ..., -7.2696e+01,
           1.4530e+02, -1.4386e+02]]),
 tensor([[2.],
         [0.],
         [1.],
         ...,
         [0.],
         [0.],
         [1.]]))

In [26]:
print('Shape of input tensor and target tensor::  ',inputs.shape, targets.shape)
print('datatype of input tensor and target tensor::  ',inputs.dtype, targets.dtype)

Shape of input tensor and target tensor::   torch.Size([7001, 9]) torch.Size([7001, 1])
datatype of input tensor and target tensor::   torch.float32 torch.float32


In [27]:
dataset = TensorDataset(inputs, targets)
dataset

In [28]:
train_ds, val_ds = train_test_split(dataset, random_state=0)
batch_size=50
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [34]:
input_size = len(x)
output_size =1# len(y)
input_size,output_size

(9, 1)

In [35]:
class WineQuality(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size,output_size) #???                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb): 
        out = self.linear(xb) #???                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out,targets) #???                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out,targets) #???                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 100th epoch
        if (epoch+1) % 100 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [36]:
model=WineQuality()

In [37]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [40]:
epochs = 1500
lr = 1e-6
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [100], val_loss: 5864.8325
Epoch [200], val_loss: 5785.3047
Epoch [300], val_loss: 5705.8413
Epoch [400], val_loss: 5626.4009
Epoch [500], val_loss: 5547.0347
Epoch [600], val_loss: 5467.8569
Epoch [700], val_loss: 5388.9019
Epoch [800], val_loss: 5309.6343
Epoch [900], val_loss: 5230.9775
Epoch [1000], val_loss: 5151.9976
Epoch [1100], val_loss: 5072.9932
Epoch [1200], val_loss: 4994.1699
Epoch [1300], val_loss: 4915.3213


KeyboardInterrupt: 

In [ ]:
plt.title('val_loss vs. No. of epochs');
loss_mat = [res['val_loss'] for res in [result] + history5]
plt.plot(loss_mat, '-x')
plt.xlabel('epoch')
plt.ylabel('val_loss')

val_loss = loss_mat[-1]